In [1]:
from polymarket_analysis.data.load_history import load_price_history
from polymarket_analysis.data.markets import load_raw_markets, download_raw_markets
from polymarket_analysis.data.binance import Binance
from polymarket_analysis.data.contract_loader import ContractLoader
import pandas as pd

raw_markets = load_raw_markets()

print(f"Loaded {len(raw_markets)} markets.")


contracts = ContractLoader().load_crypto_contracts(pd.DataFrame(raw_markets))

print(f"Loaded {len(contracts)} crypto contracts.")

Loaded 9819 markets.
Could not extract date from market slug: ageagea-958-915
Expected exactly one time in description, found 4: This market will resolve to "Yes" if both Bitcoin and Ethereum are Up on July 9. Otherwise, this market will resolve to "No".

Bitcoin will be considered Up if the "Close" price for the Binance 1 minute candle for BTC/USDT 9 Jul '25 12:00 in the ET timezone (noon) is lower than the final "Close" price for the 10 Jul '25 12:00 ET candle.

Ethereum will be considered Up if the "Close" price for the Binance 1 minute candle for ETH/USDT 9 Jul '25 12:00 in the ET timezone (noon) is lower than the final "Close" price for the 10 Jul '25 12:00 ET candle.

The resolution source for both assets is Binance, specifically the BTC/USDT and ETH/USDT pairs with 1m” and “Candles” selected on the top bar available at:
https://www.binance.com/en/trade/BTC_USDT
https://www.binance.com/en/trade/ETH_USDT

Please note that this market is about the prices according to Binance BTC/US

In [ ]:
token_histories = [load_price_history(contract) for contract in contracts]
nonempty_histories = [history for history in token_histories[0:1] if history is not None]


In [3]:
# import pickle

# with open('/Users/kate/projects/polymarket/data/polymarket/prices_2025-07-23', 'wb') as f:
#         pickle.dump(nonempty_histories, f)
    

In [14]:
from polymarket_analysis.analytics.gbm_param_estimator import GBMParameterEstimator
from polymarket_analysis.analytics.gbm_hourly_template_estimator import GBMParameterHourlyTemplateEstimator


# gbm = GBMParameterEstimator(symbol="BTC-USD", period="1y")

# gbm.fetch_data()
# # Fetch data and estimate parameters
# gbm.calculate_returns()
# params = gbm.estimate_parameters()

from polymarket_analysis.analytics import gbm_hourly_template_estimator
import importlib
importlib.reload(gbm_hourly_template_estimator)

gbm = GBMParameterHourlyTemplateEstimator()

Loaded 111585 data points from /Users/kate/projects/polymarket/data/btc_5min_data.json


In [15]:
import importlib
from polymarket_analysis.analytics import gbm_extender
importlib.reload(gbm_extender)

extender = gbm_extender.GbmExtender(gbm)

In [16]:
from polymarket_analysis.strategies import strategy
# import importlib
# importlib.reload(strategy)
from polymarket_analysis.strategies.strategy import EdgeBasedStrategy

In [17]:
min_timestamp = min(history.price_df['timestamp'].min() for history in nonempty_histories)
max_timestamp = max(history.price_df['timestamp'].max() for history in nonempty_histories)
btc_df_loop = Binance.load_bitcon_5min(from_date=min_timestamp, to_date=max_timestamp)

In [ ]:
from polymarket_analysis.strategies.strategy_backtest import StrategyBacktester
from polymarket_analysis.strategies.strategy import EdgeBasedStrategy
import pandas as pd

results_list = []
for history in nonempty_histories:
    print(f"Processing history for contract: {history.contract.title}")
    # Calculate the fair value for this history
    extender.extend_price_history(history, btc_df_loop)

    print(f"Extended price history for contract: {history.contract.title}")

    backtester = StrategyBacktester()
    trading_strategy = EdgeBasedStrategy(use_kelly = True)
    settlement_price = history.contract.get_settlement_price()

    if settlement_price is None:
        print(f"Skipping backtest for '{history.contract.title}' (outcome not set).")
        continue

    backtester.backtest(
        strategy=trading_strategy,
        market_data=history.price_df,
        final_outcome=settlement_price
    )

    # --- 3. Store the results ---
    metrics = backtester.metrics
    result_data = {
        'title': history.contract.title,
        'total_pnl': metrics.get('total_pnl'),
        'num_trades': metrics.get('num_trades'),
        'win_rate': metrics.get('win_rate'),
        'total_invested': metrics.get('total_invested'),
        'realized_pnl': metrics.get('realized_pnl'),
        # 'avg_return': metrics.get('total_pnl') / metrics.get('total_invested'),
    }
    results_list.append(result_data)

results_df = pd.DataFrame(results_list)

Processing history for contract: Bitcoin above $106,000 on July 9? - Yes


In [19]:
backup_df = results_df.copy()

In [20]:
results_df = backup_df

In [21]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

results_df['return'] = results_df['total_pnl'] / results_df['total_invested']
results_df[['title', 'total_pnl', 'num_trades', 'total_invested', 'return']]


,title,total_pnl,num_trades,total_invested,return
0,"Bitcoin above $106,000 on July 9? - Yes",None,None,None,NaN


In [22]:
print(f"Total invested: {results_df['total_invested'].sum()}")
print(f"Total PnL: {results_df['total_pnl'].sum()}")
print(f"Total PnL / Total invested: {results_df['total_pnl'].sum() / results_df['total_invested'].sum()}")

Total invested: 0
Total PnL: 0


ZeroDivisionError: division by zero

In [ ]:
token_history = nonempty_histories[10]

polymarket_start = token_history.price_df['timestamp'].min()
polymarket_end = token_history.contract.target_time

btc_df = Binance.load_bitcon_5min(
    from_date=polymarket_start, 
    to_date=polymarket_end
)
print(f"Filtered Bitcoin data between {polymarket_start}, {polymarket_end}: {len(btc_df)} rows")

extender.extend_price_history(token_history, btc_df)

Filtered Bitcoin data between 2025-07-09 00:00:07+00:00, 2025-07-09 12:00:00-04:00: 192 rows


/Users/kate/projects/polymarket/src/polymarket_analysis/analytics/gbm_param_estimator.py:157: RuntimeWarning: invalid value encountered in sqrt
  log_volatility = self.sigma * np.sqrt(days_ahead)
/Users/kate/projects/polymarket/src/polymarket_analysis/analytics/gbm_param_estimator.py:193: RuntimeWarning: invalid value encountered in sqrt
  std_dev = np.sqrt(variance)


PriceHistory(contract=TokenPriceRangeContract(title='Will the price of Bitcoin be greater than $109K on July 9? - Yes', token_id='48755184112183233720746626277251474929230276218206953154642561630893239294409', outcome='Yes', description='This market will resolve according to the final "Close" price of the Binance 1 minute candle for BTC/USDT 12:00 in the ET timezone (noon) on the date specified in the title. Otherwise, this market will resolve to "No".\n\nThe resolution source for this market is Binance, specifically the BTC/USDT "Close" prices currently available at https://www.binance.com/en/trade/BTC_USDT with "1m" and "Candles" selected on the top bar..\n\nIf the reported value falls exactly between two brackets, then this market will resolve to the higher range bracket.\n\nPlease note that this market is about the price according to Binance BTC/USDT, not according to other sources or spot markets.', end_date=Timestamp('2025-07-09 00:00:00+0000', tz='UTC'), raw_market={'enable_orde

In [ ]:
# import importlib
# from polymarket_analysis.data.model import polymarket_contract
# importlib.reload(polymarket_contract)

# from polymarket_analysis.data.model.polymarket_contract import TokenPriceRangeContract

In [ ]:
# %matplotlib tk
%matplotlib inline

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
# Create subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
        x=token_history.price_df['timestamp'], 
        y=token_history.price_df['price'],
        mode='lines+markers',
        name=token_history.contract.title.replace('$', '&#36;'),
    ))
    
# Add Bitcoin price
fig.add_trace(
    go.Scatter(
        x=btc_df['timestamp'], 
        y=btc_df['price'],
        mode='lines',
        name='Bitcoin Price',
        # line=dict(color='black'),
    ),
    secondary_y=True,
)

# Add Bitcoin estimate
fig.add_trace(
    go.Scatter(
        x=token_history.price_df['timestamp'], 
        y=token_history.price_df['fair_price'],
        mode='lines',
        name='Estimate',
        # line=dict(color='black'),
    ),
)

fig.add_trace(
    go.Scatter(
        x=token_history.price_df['timestamp'],
        y=token_history.price_df['fair_price'] - token_history.price_df['price'],
        mode='lines',
        name='Edge (Fair Price - Market Price)',
        line=dict(color='rgba(255, 0, 255, 0.7)')
    ),
    secondary_y=False
)

fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="grey", secondary_y=False)

fig.add_hline(y=token_history.contract.get_settlement_price(), line_width=1, line_dash="dash", line_color="grey", secondary_y=False)


# Update layout
fig.update_xaxes(title_text="Timestamp")
fig.update_yaxes(title_text="Polymarket Price", secondary_y=False)
fig.update_yaxes(title_text="Bitcoin Price (USD)", secondary_y=True)

fig.update_layout(
    title_text="Polymarket vs Bitcoin Price Comparison (5-minute data)",
    legend=dict(
        orientation="v",      # Display legend items in a vertical list
        yanchor="top",        # Anchor the legend's top edge
        y=-0.2,               # Position the top of the legend just below the plot area
        xanchor="left",
        x=0
    ),
    # Add a bottom margin to create enough space for both the x-axis title and the legend
    margin=dict(b=120),
    showlegend=True
)

fig.show(renderer="browser")

In [ ]:
import polymarket_analysis.strategies

from polymarket_analysis.strategies import strategy_backtest
# from polymarket_analysis.strategies import strategy

import importlib
importlib.reload(strategy_backtest)
from polymarket_analysis.strategies import strategy_backtest
# importlib.reload(strategy)

from polymarket_analysis.strategies.strategy_backtest import StrategyBacktester
from polymarket_analysis.strategies.strategy import EdgeBasedStrategy

trading_strategy = EdgeBasedStrategy(use_kelly=True)


In [ ]:
backtest = StrategyBacktester()


settlement_price = token_history.contract.get_settlement_price()
retult = backtest.backtest(trading_strategy
                           , market_data=token_history.price_df,
                           final_outcome=settlement_price
)

Skipping order generation for at 2025-07-09 00:00:07+00:00 due to NaN fair price
Processing order: buy 3.390233854239519 at 0.405 on 2025-07-09 00:20:08+00:00
Partial close: Trade(entry_timestamp=Timestamp('2025-07-09 00:20:08+0000', tz='UTC'), entry_price=0.405, entry_volume=1.0566911582615455, side=<OrderSide.BUY: 'buy'>, exit_timestamp=Timestamp('2025-07-09 03:10:07+0000', tz='UTC'), exit_price=0.31, realized_pnl=-0.10038566003484685, unrealized_pnl=None)
Processing order: sell 1.0566911582615455 at 0.31 on 2025-07-09 03:10:07+00:00
Processing order: buy 1.4509555645344379 at 0.245 on 2025-07-09 03:20:07+00:00
Partial close: Trade(entry_timestamp=Timestamp('2025-07-09 00:20:08+0000', tz='UTC'), entry_price=0.405, entry_volume=1.053041420851776, side=<OrderSide.BUY: 'buy'>, exit_timestamp=Timestamp('2025-07-09 04:40:09+0000', tz='UTC'), exit_price=0.27, realized_pnl=-0.14216059181498977, unrealized_pnl=None)
Processing order: sell 1.053041420851776 at 0.27 on 2025-07-09 04:40:09+00:0

In [ ]:
backtest.metrics

{'total_pnl': 8.43826920004387,
 'realized_pnl': 8.43826920004387,
 'num_trades': 22,
 'num_winning_trades': 13,
 'num_losing_trades': 3,
 'total_invested': 53.46502876273035,
 'win_rate': 0.8125}

In [ ]:
retult

,timestamp,position,unrealized_pnl,realized_pnl,total_pnl,cash,total_value,current_price
0,2025-07-09 00:20:08+00:00,3.390234,0.000000,0.000000,0.000000,-1.373045,0.000000,0.4050
1,2025-07-09 03:10:07+00:00,2.333543,-0.221687,-0.100386,-0.322072,-1.045470,-0.322072,0.3100
2,2025-07-09 03:20:07+00:00,3.784498,-0.373367,-0.100386,-0.473752,-1.400955,-0.473752,0.2450
3,2025-07-09 04:40:09+00:00,2.731457,-0.136594,-0.242546,-0.379140,-1.116633,-0.379140,0.2700
4,2025-07-09 05:40:07+00:00,1.317255,0.191002,-0.242367,-0.051365,-0.565095,-0.051365,0.3900
5,2025-07-09 06:00:08+00:00,0.227119,0.028390,-0.106100,-0.077710,-0.161744,-0.077710,0.3700
6,2025-07-09 06:50:07+00:00,1.426915,0.023848,-0.106100,-0.082253,-0.581673,-0.082253,0.3500
7,2025-07-09 07:00:08+00:00,-0.355826,0.000000,-0.010907,-0.010907,0.131423,-0.010907,0.4000
8,2025-07-09 08:20:07+00:00,-1.433139,0.000000,-0.010907,-0.010907,0.546189,-0.005570,0.3850
9,2025-07-09 09:50:07+00:00,-2.865360,0.000000,-0.010907,-0.010907,1.018822,0.073253,0.3300
